### Create Toronto Neighborhoods DataFrame

In [1]:
import pandas as pd
import numpy as np
!conda install lxml --yes
import lxml # library to read html

!conda install -c conda-forge geopy --yes # install geopy to pull coordinates
from geopy.geocoders import Nominatim

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         156 KB

The following packages will be UPDATED:

    certifi: 2019.6.16-py36_1  conda-forge --> 2019.6.16-py36_1 
    openssl: 1.1.1c-h516909a_0 conda-forge --> 1.1.1c-h7b6447c_1


certifi-2019.6.16    | 156 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4

Load Toronto neighborhood list from wikipedia.

In [2]:
url='https://en.wikipedia.org/wiki/List_of_city-designated_neighbourhoods_in_Toronto'
dfs=pd.read_html(url)
TOR_Neigh=dfs[0]
print(TOR_Neigh.shape)
TOR_Neigh.head()

(140, 5)


,CDN number,City-designated area,Former city/borough,Neighbourhoods covered,Map
0,129,Agincourt North,Scarborough,Agincourt and Brimwood,NaN
1,128,Agincourt South-Malvern West,Scarborough,Agincourt and Malvern,NaN
2,20,Alderwood,Etobicoke,Alderwood,NaN
3,95,Annex,Old City of Toronto,The Annex and Seaton Village,NaN
4,42,Banbury-Don Mills,North York,Don Mills,NaN


Clean up table. Remove columns that are not needed.

In [3]:
TOR_Neigh=TOR_Neigh[['Former city/borough','City-designated area']]
TOR_Neigh.columns=['Borough','Neighborhood']
TOR_Neigh.head()

,Borough,Neighborhood
0,Scarborough,Agincourt North
1,Scarborough,Agincourt South-Malvern West
2,Etobicoke,Alderwood
3,Old City of Toronto,Annex
4,North York,Banbury-Don Mills


Add coordinates to each neighborhood.

In [5]:
# Pull coordinates
TOR_Neigh.insert(2,'Lat',np.nan)
TOR_Neigh.insert(3,'Long',np.nan)
geolocator=Nominatim(user_agent='TOR')

for i in range(len(TOR_Neigh)):
    try:
        address=TOR_Neigh.iloc[i,1]+', '+TOR_Neigh.iloc[i,0]
        location=geolocator.geocode(address)
        TOR_Neigh.iloc[i,2]=location.latitude
        TOR_Neigh.iloc[i,3]=location.longitude
    except:
        try:
            address=TOR_Neigh.iloc[i,1]+', Toronto'
            location=geolocator.geocode(address)
            TOR_Neigh.iloc[i,2]=location.latitude
            TOR_Neigh.iloc[i,3]=location.longitude
        except:
            try:
                address=TOR_Neigh.iloc[i,1].split('-')[0]+', '+TOR_Neigh.iloc[i,0]
                location=geolocator.geocode(address)
                TOR_Neigh.iloc[i,2]=location.latitude
                TOR_Neigh.iloc[i,3]=location.longitude
            except:
                try:
                    address=TOR_Neigh.iloc[i,1].split('-')[1]+', '+TOR_Neigh.iloc[i,0]
                    location=geolocator.geocode(address)
                    TOR_Neigh.iloc[i,2]=location.latitude
                    TOR_Neigh.iloc[i,3]=location.longitude
                except:
                    try:
                        address=TOR_Neigh.iloc[i,1].split(' ')[1]+', '+TOR_Neigh.iloc[i,0]
                        location=geolocator.geocode(address)
                        TOR_Neigh.iloc[i,2]=location.latitude
                        TOR_Neigh.iloc[i,3]=location.longitude
                    except:
                        None
                        
    
TOR_Neigh.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Borough,Neighborhood,Lat,Long
0,Scarborough,Agincourt North,43.808038,-79.266439
1,Scarborough,Agincourt South-Malvern West,43.775718,-79.259021
2,Etobicoke,Alderwood,43.601717,-79.545232
3,Old City of Toronto,Annex,43.670338,-79.407117
4,North York,Banbury-Don Mills,43.734804,-79.357243


Select boroughs that contain the word 'Toronto'

In [6]:
TOR_Neigh_s=TOR_Neigh[TOR_Neigh['Borough'].str.contains('Toronto')]
TOR_Neigh_s.shape

(44, 4)

In [7]:
# validate if there is any error in the selected dataset
TOR_Neigh_s.isna().sum()

Borough         0
Neighborhood    0
Lat             0
Long            0
dtype: int64

### Explore Neighborhoods in Toronto

Define FourSquare credential

In [8]:
# @hidden
client_id='GYU440FFLMNB1IBZLBCVSMUJCOXHFPDVUUDB4W5UJNVVRLWN'
client_secret='2JUAIXJV1DY5ODZRIE2GI4RYHYHR12KSGLDB21OFCSAJSAAJ'
version='20190907'


Define a function to pull venue data from FourSquare.

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, long in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            long, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            long, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
limit=100
r=1000

TOR_venues = getNearbyVenues(names=TOR_Neigh_s['Neighborhood'],
                                   latitudes=TOR_Neigh_s['Lat'],
                                   longitudes=TOR_Neigh_s['Long'],
                                   radius=r
                                )

Annex
Bay Street Corridor
Blake-Jones
Cabbagetown-South St. James Town
Casa Loma
Church-Yonge Corridor
Corso Italia-Davenport
Danforth Village - Toronto
Dovercourt-Wallace Emerson-Junction
Dufferin Grove
East End-Danforth
Forest Hill North
Forest Hill South
Greenwood-Coxwell
High Park North
High Park-Swansea
Junction Area
Kensington-Chinatown
Lawrence Park North
Lawrence Park South
Little Portugal
Moss Park
Mount Pleasant East
Mount Pleasant West
Niagara
North Riverdale
North St. James Town
Palmerston-Little Italy
Parkdale
Playter Estates-Danforth
Regent Park
Roncesvalles
Rosedale-Moore Park
Runnymede-Bloor West Village
South Riverdale
The Beaches
Trinity–Bellwoods
University
Waterfront Communities-The Island
Weston-Pellam Park
Woodbine Corridor
Wychwood
Yonge and Eglinton
Yonge-St.Clair


In [27]:
print(TOR_venues.shape)
TOR_venues.head()

(3405, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Annex,43.670338,-79.407117,Jean Sibelius Square,43.671426,-79.408831,Park
1,Annex,43.670338,-79.407117,Fresh on Bloor,43.666755,-79.403491,Vegetarian / Vegan Restaurant
2,Annex,43.670338,-79.407117,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
3,Annex,43.670338,-79.407117,Fuwa Fuwa Japanese Pancakes,43.665880,-79.407840,Pastry Shop
4,Annex,43.670338,-79.407117,BMV Books,43.665860,-79.406824,Bookstore


Show venue categories in columns.

In [28]:
# one hot encoding
TOR_onehot = pd.get_dummies(TOR_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
TOR_onehot['Neighborhood'] = TOR_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [TOR_onehot.columns[-1]] + list(TOR_onehot.columns[:-1])
TOR_onehot = TOR_onehot[fixed_columns]

TOR_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,Art Gallery,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group by neighborhood. Estimate the means of frequency of each category.

In [29]:
TOR_grouped = TOR_onehot.groupby('Neighborhood').mean().reset_index()
TOR_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Annex,0.000000,0.0,0.00,0.0,0.010000,0.0,0.0,0.0,0.0,...,0.000000,0.030000,0.00,0.01,0.010000,0.0,0.0,0.000000,0.000000,0.00
1,Bay Street Corridor,0.010000,0.0,0.01,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.01,0.01,0.010000,0.0,0.0,0.000000,0.010000,0.01
2,Blake-Jones,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.015625,0.000000,0.00,0.00,0.046875,0.0,0.0,0.000000,0.015625,0.00
3,Cabbagetown-South St. James Town,0.000000,0.0,0.00,0.0,0.011905,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00,0.00,0.011905,0.0,0.0,0.011905,0.000000,0.00
4,Casa Loma,0.012048,0.0,0.00,0.0,0.012048,0.0,0.0,0.0,0.0,...,0.000000,0.024096,0.00,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.00


Identify the most popular venues.

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = TOR_grouped['Neighborhood']

for ind in np.arange(TOR_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(TOR_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(44, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Annex,Café,Restaurant,Bakery,Italian Restaurant,Pizza Place,Korean Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Dessert Shop
1,Bay Street Corridor,Coffee Shop,Italian Restaurant,Park,Sushi Restaurant,Burger Joint,Café,Japanese Restaurant,Ramen Restaurant,Dance Studio,Ice Cream Shop
2,Blake-Jones,Korean Restaurant,Bubble Tea Shop,Coffee Shop,Café,Park,Vietnamese Restaurant,Dessert Shop,Pizza Place,Sandwich Place,Bank
3,Cabbagetown-South St. James Town,Coffee Shop,Diner,Gay Bar,Park,Thai Restaurant,Gastropub,Restaurant,Café,Japanese Restaurant,Ramen Restaurant
4,Casa Loma,Coffee Shop,Sandwich Place,Pizza Place,Park,History Museum,Mexican Restaurant,Café,French Restaurant,Vegetarian / Vegan Restaurant,Jewish Restaurant


### Cluster Neighborhoods

Run K-means to cluster the neighborhoods into 4 clusters.

In [47]:
# set number of clusters
kclusters = 4

TOR_grouped_clustering = TOR_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TOR_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1], dtype=int32)

In [48]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

TOR_merged = TOR_Neigh_s

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
TOR_merged = TOR_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

TOR_merged.head() # check the last columns!

,Borough,Neighborhood,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Old City of Toronto,Annex,43.670338,-79.407117,0,Café,Restaurant,Bakery,Italian Restaurant,Pizza Place,Korean Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Dessert Shop
6,Old City of Toronto,Bay Street Corridor,43.664286,-79.387114,0,Coffee Shop,Italian Restaurant,Park,Sushi Restaurant,Burger Joint,Café,Japanese Restaurant,Ramen Restaurant,Dance Studio,Ice Cream Shop
14,Old City of Toronto,Blake-Jones,43.781933,-79.423086,0,Korean Restaurant,Bubble Tea Shop,Coffee Shop,Café,Park,Vietnamese Restaurant,Dessert Shop,Pizza Place,Sandwich Place,Bank
19,Old City of Toronto,Cabbagetown-South St. James Town,43.666068,-79.370842,0,Coffee Shop,Diner,Gay Bar,Park,Thai Restaurant,Gastropub,Restaurant,Café,Japanese Restaurant,Ramen Restaurant
21,Old City of Toronto,Casa Loma,43.678101,-79.409416,0,Coffee Shop,Sandwich Place,Pizza Place,Park,History Museum,Mexican Restaurant,Café,French Restaurant,Vegetarian / Vegan Restaurant,Jewish Restaurant


In [49]:
# create map using Toronto coordinates
latitude=43.6432
longitude=-79.3832
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(TOR_merged['Lat'], TOR_merged['Long'], TOR_merged['Neighborhood'], TOR_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters